In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import re
import requests
import json
import os


In [10]:
# read in all gene names
gene_list_raw =pd.read_csv("230323_EST_ENSG_GENE_new.csv")
gene_list_raw = gene_list_raw.rename(columns={'gene':'SYMBOL'})
gene_list_raw

,index,ENST,ENSG,protein,SYMBOL,Uniprot
0,0,ENST00000646641,ENSG00000267534,s1pr2_human,S1PR2,O95136
1,121,ENST00000547270,ENSG00000257138,t2r38_human,TAS2R38,P59533
2,219,ENST00000390675,ENSG00000256436,t2r31_human,TAS2R31,P59538
3,344,ENST00000539585,ENSG00000256188,t2r30_human,TAS2R30,P59541
4,483,ENST00000538986,ENSG00000255837,t2r20_human,TAS2R20,P59543
...,...,...,...,...,...,...
395,64029,ENST00000510937,ENSG00000226306,npy6r_human,NPY6R,Q99463
396,64030,ENST00000641193,ENSG00000279301,o2t11_human,OR2T11,Q8NH01
397,64031,ENST00000641732,ENSG00000172146,or1a1_human,OR1A1,Q9P1Q5
398,64032,ENST00000328890,ENSG00000183024,or1g1_human,OR1G1,P47890


In [11]:
VEPwithGB_icd10 = pd.read_csv('VEPwithGB_icd10_0323.csv', index_col = None)
VEPwithGB_icd10['mutation_from'] = VEPwithGB_icd10['Amino_acids'].str.split('/',expand=True)[0]
VEPwithGB_icd10['mutation_to'] = VEPwithGB_icd10['Amino_acids'].str.split('/',expand=True)[1]
VEPwithGB_icd10_uniid = pd.merge(VEPwithGB_icd10,gene_list_raw[['protein','SYMBOL','Uniprot']],on='SYMBOL',how='left')
unq_VEPwithGB_icd10 = VEPwithGB_icd10_uniid.drop_duplicates(subset=['SYMBOL','Protein_position','mutation_from','mutation_to'], keep='first')
unq_VEPwithGB_icd10 = unq_VEPwithGB_icd10.reset_index()
unq_VEPwithGB_icd10

/Users/joannahench/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (8,9,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,index,Location,Allele,Protein_position,SYMBOL,Amino_acids,locus,SIFT,PolyPhen,BayesDel_addAF_score,...,BETA,SE,AF.Cases,AF.Controls,Pvalue,vcf,mutation_from,mutation_to,protein,Uniprot
0,0,11:113412715-113412715,C,327,DRD2,K/E,chr11:113412715,tolerated(0.27),benign(0.031),-0.260005,...,0.43126,0.53696,3.472200e-03,0.002413,0.421890,11 113412715 . T C,K,E,drd2_human,P14416
1,45,11:113412762-113412762,C,311,DRD2,S/C,chr11:113412762,deleterious(0.05),possibly_damaging(0.908),-0.393485,...,-0.14869,0.18710,1.736100e-02,0.020282,0.426780,11 113412762 . G C,S,C,drd2_human,P14416
2,90,11:113412766-113412766,A,310,DRD2,P/S,chr11:113412766,tolerated(0.61),benign(0.001),-0.399698,...,-1.00910,1.63230,0.000000e+00,0.000298,0.536450,11 113412766 . G A,P,S,drd2_human,P14416
3,135,11:113416935-113416935,T,154,DRD2,V/I,chr11:113416935,tolerated(0.16),possibly_damaging(0.767),0.134914,...,-1.00480,6.30530,0.000000e+00,0.000015,0.873390,11 113416935 . C T,V,I,drd2_human,P14416
4,180,13:77901095-77901095,T,305,EDNRB,S/N,chr13:77901095,tolerated(0.92),benign(0.006),-0.601516,...,-0.42015,0.22281,8.333300e-03,0.014423,0.059333,13 77901095 . C T,S,N,ednrb_human,P24530
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15486,654517,X:114907377-114907377,T,447,HTR2C,P/S,chrX:114907377,tolerated(1),benign(0.035),-0.203906,...,-0.56465,9.73350,0.000000e+00,0.000005,0.953740,X 114907377 . C T,P,S,5ht2c_human,P28335
15487,654562,X:114907381-114907381,T,448,HTR2C,S/F,chrX:114907381,tolerated(0.2),benign(0.194),-0.218816,...,-1.00070,19.87100,0.000000e+00,0.000006,0.959840,X 114907381 . C T,S,F,5ht2c_human,P28335
15488,654605,X:114907387-114907387,C,450,HTR2C,V/A,chrX:114907387,tolerated(0.27),benign(0.052),-0.175516,...,-0.63211,6.53650,4.941800e-07,0.000015,0.922960,X 114907387 . T C,V,A,5ht2c_human,P28335
15489,654650,X:114907399-114907399,A,454,HTR2C,R/K,chrX:114907399,tolerated(0.67),benign(0.048),-0.281812,...,-0.55746,2.88220,3.568200e-06,0.000045,0.846640,X 114907399 . G A,R,K,5ht2c_human,P28335


In [13]:
uniid_list = np.unique(unq_VEPwithGB_icd10['Uniprot'])
uniid_list

array(['P07550', 'P08173', 'P08588', 'P08908', 'P08912', 'P08913',
       'P0DMS8', 'P11229', 'P13945', 'P14416', 'P16473', 'P18089',
       'P20309', 'P21728', 'P21917', 'P21918', 'P22888', 'P23945',
       'P24530', 'P25021', 'P25024', 'P25025', 'P25100', 'P28221',
       'P28222', 'P28223', 'P28335', 'P29274', 'P29275', 'P30542',
       'P32241', 'P32245', 'P32246', 'P34969', 'P34972', 'P34998',
       'P35348', 'P35367', 'P35368', 'P35462', 'P41180', 'P41231',
       'P41587', 'P41594', 'P41595', 'P41597', 'P43220', 'P47871',
       'P47872', 'P47898', 'P48546', 'P49190', 'P49682', 'P50406',
       'P51677', 'P51681', 'P51685', 'P61073', 'Q02643', 'Q03431',
       'Q13255', 'Q13639', 'Q14416', 'Q16602', 'Q7RTX0', 'Q9H244',
       'Q9H3N8', 'Q9Y5N1'], dtype=object)

# for each GPCR, get AF pdb file

In [14]:
# a function to get the alphafold structure (pdb file) for each gene
def getAF(uniprot_id):
    # Set the API endpoint and the UniProt ID of the protein
    endpoint = "https://alphafold.ebi.ac.uk/files"

    # Send a GET request to the API to retrieve the protein's structure
    response = requests.get(f"{endpoint}/AF-{uniprot_id}-F1-model_v4.pdb")

    # Check the status code of the response to ensure it was successful
    if response.status_code == 200:
        filename = f"/Users/joannahench/Downloads/foldx5MacStd.tar_/Genebass/{uniprot_id}/{uniprot_id}.pdb"
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        # Write the contents of the response to a PDB file
        with open(filename, "w") as f:
            f.write(response.text)
    else:
        print(f"Failed to retrieve structure for {uniprot_id}: {response.status_code}")



In [15]:
#get the structure
for i in range(len(uniid_list)):
    getAF(uniid_list[i])

# make mutation list

In [20]:

for i in range(unq_VEPwithGB_icd10.shape[0]):
    # get the uniprot id according to entryname and convert the Series to a string with index excluded
    uniprot_id = unq_VEPwithGB_icd10['Uniprot'][i]
    # gene = gene_list_raw.loc[gene_list_raw['protein']==gpcr_valid_missense.iloc[i,4],'gene'].to_string(index=False)
    # uniid_list.append(uniprot_id)
    # gene_list.append(gene)
    mut_info = unq_VEPwithGB_icd10.iloc[i,64]+'A'+str(unq_VEPwithGB_icd10.iloc[i,3])+unq_VEPwithGB_icd10.iloc[i,65]
    with open(f"/Users/joannahench/Downloads/foldx5MacStd.tar_/Genebass/{uniprot_id}/individual_list_{uniprot_id}.txt", 'a') as file:
        # write a new line to the file
        file.write(mut_info+';'+"\n")
    

# make bash file

In [21]:
# generate the bash file
for i in range(len(uniid_list)):
        uniprot_id = uniid_list[i]
        with open("foldx_command.sh", 'a') as foldx:
                foldx.write(f'cd {uniprot_id}'+"\n")
                # write the command
                foldx.write(f"../../foldx --command=BuildModel --pdb={uniprot_id}.pdb --mutant-file=individual_list_{uniprot_id}.txt"+"\n")
                foldx.write('cd ..'+"\n") 

In [ ]:
../../foldx --help --command=BuildModel

In [ ]:
../../foldx --command=BuildModel --pdb=/Users/joannahench/Downloads/foldx5MacStd.tar_/GPCRdb/P0DMS8/P0DMS8.pdb --mutant-file=/Users/joannahench/Downloads/foldx5MacStd.tar_/GPCRdb/P0DMS8/individual_list_P0DMS8.txt

In [ ]:
nohup ./foldx_test.sh &